# 151: Swissmetro Panel Latent Class

In [ ]:
# TEST
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)
np.set_printoptions(precision=12)

from pytest import approx

import larch as lx

In [ ]:
import numpy as np
import pandas as pd

import larch as lx
from larch import P, X

print(lx.versions())

This example is a latent class mode choice model (also known as a discrete mixture model) built using the Swissmetro example dataset.
This model is similar to the simple latent class model shown in [Example 107](./107-swissmetro-latent-class). The key difference is
that the class membership is determined per-respondent instead of per-observation. The underlying data is from a stated preference
survey, where respondents made repeated choices with various different alternative attributes.

As before, the first step is to load the data and do some pre-processing, so that the format and scale of the 
data matches that from the Biogeme example.

In [ ]:
raw = pd.read_csv(lx.example_file("swissmetro.csv.gz"))
raw["SM_COST"] = raw["SM_CO"] * (raw["GA"] == 0)
raw["TRAIN_COST"] = raw.eval("TRAIN_CO * (GA == 0)")
raw["TRAIN_COST_SCALED"] = raw["TRAIN_COST"] / 100
raw["TRAIN_TT_SCALED"] = raw["TRAIN_TT"] / 100
raw["SM_COST_SCALED"] = raw.eval("SM_COST / 100")
raw["SM_TT_SCALED"] = raw["SM_TT"] / 100
raw["CAR_CO_SCALED"] = raw["CAR_CO"] / 100
raw["CAR_TT_SCALED"] = raw["CAR_TT"] / 100
raw["CAR_AV_SP"] = raw.eval("CAR_AV * (SP!=0)")
raw["TRAIN_AV_SP"] = raw.eval("TRAIN_AV * (SP!=0)")

In [ ]:
raw

We now have a pandas DataFrame in "wide" or [idco](idco) format, with one row per choice observation.
For this example, it is relevant to note there is more than one row per respondent, and the various
respondents can be identified by the "ID" column.

We can convert this to a Larch Dataset, filtering to include only the cases of interest,
following the same filter applied in the Biogeme example.

In [ ]:
data = lx.Dataset.construct.from_idco(raw).dc.query_cases(
    "PURPOSE in (1,3) and CHOICE != 0"
)

The result is a Dataset with 6,768 cases.

In [ ]:
data

For convenience, we can collect the names of the variables that define alternative availability
into a dictionary, keyed on the codes we will use to represent each alternative.

In [ ]:
availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}

Then we can contruct choice models for each of the classes.  This is done in the
usual manner for Larch choice models, assigning utility functions, alternative 
availability conditions, and choice markers in the usual manner.

In this example, Class 1 chooses based on cost, and the utility functions include
a set of alternative specific constants.

In [ ]:
m1 = lx.Model(title="Model1")
m1.availability_co_vars = availability_co_vars
m1.choice_co_code = "CHOICE"
m1.utility_co[1] = P("ASC_TRAIN") + X("TRAIN_COST_SCALED") * P("B_COST")
m1.utility_co[2] = X("SM_COST_SCALED") * P("B_COST")
m1.utility_co[3] = P("ASC_CAR") + X("CAR_CO_SCALED") * P("B_COST")

Class 2 is the nearly same model, defined completely seperately but in a similar manner. 
The only difference is in the utility functions, which adds travel time to the utility.

In [ ]:
m2 = lx.Model(title="Model2")
m2.availability_co_vars = availability_co_vars
m2.choice_co_code = "CHOICE"
m2.utility_co[1] = (
    P("ASC_TRAIN")
    + X("TRAIN_TT_SCALED") * P("B_TIME")
    + X("TRAIN_COST_SCALED") * P("B_COST")
)
m2.utility_co[2] = X("SM_TT_SCALED") * P("B_TIME") + X("SM_COST_SCALED") * P("B_COST")
m2.utility_co[3] = (
    P("ASC_CAR") + X("CAR_TT_SCALED") * P("B_TIME") + X("CAR_CO_SCALED") * P("B_COST")
)

For this example, we will also add a third choice class.
Class 3 represents a lexicographic or non-compensatory choice system, where 
the choice of mode is based exclusively on cost, without regard to any other
choice attribute. When two modes have the same cost, they are considered to be
equally likely.  This lexicographic is still represented in the same mathematical 
manner as the other classes that employ typical multinomial logit models, except 
we will constrain the `Z_COST` parameter to enforce the desired choice rule for this
class.

The use of this kind of choice model in a latent class structure can be challenging 
with cross-sectional data, as it is very easy to create over-specified models, i.e.
it is hard to tell from one choice observation whether a decision maker is choosing
based on a simple choice rule or considering trade-offs between various attributes.
In a panel data study as shown in this example, it is much easier to mathematically 
disambiguate between compensatory and non-compensatory choice processes.

In [ ]:
m3 = lx.Model(title="Model3")
m3.availability_co_vars = availability_co_vars
m3.choice_co_code = "CHOICE"
m3.utility_co[1] = X("TRAIN_COST_SCALED") * P("Z_COST")
m3.utility_co[2] = X("SM_COST_SCALED") * P("Z_COST")
m3.utility_co[3] = X("CAR_CO_SCALED") * P("Z_COST")

The class membership model for this latent class model is relatively simple, just constants for each class.
One class (the first one, which we label as model 101) is a reference point, and the other classes are 
evaluated as more or less likely than the reference, similar to a typical MNL model.

In [ ]:
mk = lx.Model()
mk.utility_co[102] = P("W_OTHER")
mk.utility_co[103] = P("W_COST")

Finally, we assembled all the component models into one LatentClass structure.  We add the `groupid` 
argument to indicate how our panel data is structured.

In [ ]:
b = lx.LatentClass(
    mk,
    {101: m1, 102: m2, 103: m3},
    datatree=data.dc.set_altids([1, 2, 3]),
    groupid="ID",
)

After creating the `LatentClass` structure, we can add contraints on parameters, such as that
needed for the lexicographic choice class.

In [ ]:
b.lock_value(Z_COST=-10000)

In [ ]:
b.set_cap(25)

In [ ]:
# TEST
assert b.loglike() == approx(-6867.245, rel=1e-4)

In [ ]:
# TEST
assert b.d_loglike() == approx(
    [
        -1.104770e02,
        -1.545916e03,
        -2.188546e01,
        -9.183448e02,
        -1.658521e02,
        8.292606e01,
        4.470348e-08,
    ],
    rel=1e-5,
)

In [ ]:
result = b.maximize_loglike(method="slsqp")

In [ ]:
b.calculate_parameter_covariance();

In [ ]:
b.parameter_summary()

In [ ]:
# TEST
assert result.loglike == approx(-4474.478515625, rel=1e-5)
assert b.pstderr == approx(
    np.array([0.048158, 0.069796, 0.069555, 0.106282, 0.161079, 0.11945, 0.0]), rel=5e-3
)
assert b.pvals == approx(
    np.array(
        [
            6.079781e-02,
            -9.362056e-01,
            -1.159657e00,
            -3.095285e00,
            -7.734768e-01,
            1.155985e00,
            -1.000000e04,
        ]
    ),
    rel=5e-3,
)